# Exploitation des jeux de données du hackathon IA & Mobilités 2024

<!> Ce guide est destiné aux participants du hackathon et est valable pour les services instanciés dans le groupe "dlb-hackathon" <!>

## Installation du client Minio S3 (stockage distribué)

In [1]:
!pip install s3fs

## Listing des fichiers avec le client S3

In [3]:
import os
import s3fs

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [4]:
fs.ls("dlb-hackathon/datasets-diffusion/reclamations_formulaire_web", refresh=True)

['dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/.keep',
 'dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/idfm_reclamations_formulaire_web.parquet']

## Dépôt de fichier sur un répertoire d'équipe avec la ligne de commande

In [18]:
# On peut uploader par une commande terminal
!mc cp ./arrets-lignes.geojson s3/dlb-hackathon/equipe-0/arrets-lignes.geojson

...es.geojson: 24.56 MiB / 24.56 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 106.90 MiB/s 0s

## Requêtage de données avec Duckdb

### Paramétrage de duckdb depuis les variables d'environnement

In [32]:
# Paramétrage pour un appel Duckdb
import duckdb as ddb
ddb.execute("SET s3_region='fr-central';")
ddb.execute("SET s3_url_style='path';")
ddb.execute("SET s3_endpoint='minio.data-platform-self-service.net';")
ddb.execute("SET s3_access_key_id='xxxx' ;") # A récupérer dans les paramètres "Valeurs de Helm" du service
ddb.execute("SET s3_secret_access_key='xxxx';") # A récupérer dans les paramètres "Valeurs de Helm" du service

### Requêtage d'un fichier parquet

In [36]:
ddb.sql("""
select * from "s3://dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/idfm_reclamations_formulaire_web.parquet" 
""").to_df().sample(5)

,id,date_demande,canal_demande,texte_anonymise,classification_services_idfm
18002,32056,2023-05-22 17:04:00,Formulaire Web,"Bonjour, \n \n J'ai voulu charger mon pass Na...",Tarification Abonnement > Support Canal Mobile...
9838,21477,2022-11-25 19:40:00,Formulaire Web,"Bonjour, \n \n J’ai effectué cette semaine de...",SAV > Attestation de paiement
13060,31999,2023-05-21 11:46:00,Formulaire Web,"Bonjour, \n Je viens vers vous pour faire un ...",Tarification Abonnement > Support Autre > Pass...
6066,33748,2023-07-10 12:08:00,Formulaire Web,"L'application m'informe que mon téléphone ""Sam...",Tarification Abonnement > Support Canal Mobile...
4387,44774,2024-05-28 15:27:00,Formulaire Web,"Bonjour, \n \n Je vous transmets ma bourse en...",Tarification Abonnement > Forfaits Imagine R >...


## Manipulation des données avec Pandas

In [10]:
!pip install pandas

### Lecture d'un fichier

In [37]:
import pandas as pd

BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "datasets-diffusion/ascenseurs_historique_etat/RELEVES_ETATS_ASCENSEURS_SNCF_RATP_2021-2024.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_csv(file_in)

/tmp/ipykernel_13902/2968963640.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in)


In [8]:
df.head()

,operateur,date_releve,code_appareil,code_fonctionnel,etat,date_etat_debut,date_etat_fin,emplacement_gare,emplacement_situation,emplacement_direction
0,RATP,2021-01-01 08:07:57,1004.0,ASC01-EQ11-A005-000000269,2.0,01/01/2021,NaN,SAINT OUEN,SA200,NaN
1,RATP,2021-01-01 08:07:57,1005.0,ASC02-EQ11-A005-000000270,2.0,01/01/2021,02/01/2021,SAINT OUEN,SA200,NaN
2,RATP,2021-01-01 08:07:57,1006.0,ASC03-EQ11-A005-000000271,2.0,01/01/2021,NaN,SAINT OUEN,SA200,NaN
3,RATP,2021-01-01 08:07:57,1007.0,ASC04-EQ11-A005-000000272,2.0,01/01/2021,NaN,SAINT OUEN,SA200,NaN
4,RATP,2021-01-01 08:07:57,1008.0,ASC05-EQ11-A005-000000273,2.0,01/01/2021,02/01/2021,SAINT OUEN,SA200,NaN


### Ecriture d'un fichier

In [40]:
BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "datasets-diffusion/donnees_territoire_GPSEO/bornes_recharges/IRVE_CU.parquet"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_parquet(file_in)

In [41]:
# Ecrire
BUCKET_OUT = "dlb-hackathon"
FILE_KEY_OUT_S3 = "equipe-0/donnees_bornes_recharges.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df.to_csv(file_out)

In [30]:
# Verification
BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "equipe-0/donnees_bornes_recharges.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_csv(file_in)

df.head()

,Unnamed: 0,objectid,adresse_ir,ville,nom_statio,geom
0,0,4,Avenue De La Division Leclerc,AUBERGENVILLE,AUBERGENVILLE - Avenue De La Division Leclerc,"b""\x01\x01\x00\x00\x00L\x83\xc0\x16\x8e\xb6\xf..."
1,1,5,Place François Mitterrand,AUBERGENVILLE,AUBERGENVILLE - Place François Mitterrand,b'\x01\x01\x00\x00\x00P\xe6\xbb\xa2\xf8\x93\xf...
2,2,6,Place De La Mairie,AULNAY-SUR-MAULDRE,AULNAY-SUR-MAULDRE - Place De La Mairie,b'\x01\x01\x00\x00\x00m\x17\xd1$\xa0\x80\xfd?\...
3,3,7,2 Rue De Charnelles,BOUAFLE,BOUAFLE - Rue De Charnelles,b'\x01\x01\x00\x00\x00\xc3@\xe9\xd5\x93T\xfe?\...
4,4,8,Rue Du Béarn,BUCHELAY,BUCHELAY - Rue Du Béarn,"b""\x01\x01\x00\x00\x00'\xf2PN,\xab\xfa?k,j\xb3..."
